# Setup

In [1]:
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import numpy as np
import json
import torch.nn as nn
import os
import utils

In [51]:
# sudo modprobe nvidia_uvm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [111]:
DATASET_NAME = "DATASET_CNN"
MODEL_NAME = 'DeeperCNN'

In [112]:
DATASET_PATH = f'../segments/{DATASET_NAME}'
TRAIN_PATH = f"{DATASET_PATH}/train"
TEST_PATH = f"{DATASET_PATH}/test"
MODEL_PATH = f'./models/{MODEL_NAME}'

# Load the model

In [113]:
mappings = utils.get_mappings(TEST_PATH)

In [114]:
model_class = utils.load_model_class(MODEL_NAME)
model = model_class(len(mappings))
model.to(device)
saving_path = f'models/{MODEL_NAME}/checkpoint.pth'
checkpoint = torch.load(saving_path)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [115]:
with open(f"utils/{DATASET_NAME}/dataset_config.json") as f:
    dataset_config = json.load(f)

# Custom Thresholds Creation

In [116]:
from collections import defaultdict
from sklearn.metrics import f1_score

def calculate_conf_scores(valid_loader, model, mappings):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    model.to(device)
    
    conf_scores = defaultdict(list)

    with torch.no_grad():
        for mel_spec, _, file_path in valid_loader:
            mel_spec = mel_spec.to(device)

            # Estraggo la specie corretta dal path
            correct_species = file_path[0].split("/")[-2]
            outputs = model(mel_spec)
            probs = torch.sigmoid(outputs)[0].cpu().numpy()

            for i, prob in enumerate(probs):
                species_name = list(mappings.keys())[i]
                is_correct = (species_name == correct_species)
                conf_scores[species_name].append((prob, is_correct))

    return conf_scores

# Step 2: Trova la soglia migliore per ciascuna specie
def compute_best_thresholds(conf_scores, num_thresholds=100):
    thresholds = {}

    for species, values in conf_scores.items():
        probs, truths = zip(*values)
        probs = np.array(probs)
        truths = np.array(truths).astype(int)

        best_thresh = 0.5
        best_f1 = 0.0

        for thresh in np.linspace(0, 1, num_thresholds):
            preds = (probs >= thresh).astype(int)
            f1 = f1_score(truths, preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh

        thresholds[species] = best_thresh
        print(f"📊 Specie: {species}, Best Threshold: {best_thresh:.3f}, F1-score: {best_f1:.3f}")

    return thresholds



In [117]:
valid_loader = utils.get_dataloader(dataset_config, split="valid", batch_size=1)
conf_scores = calculate_conf_scores(valid_loader, model, dataset_config["mappings"])
best_thresholds = compute_best_thresholds(conf_scores)

📊 Specie: Regulus ignicapilla_Common Firecrest, Best Threshold: 0.030, F1-score: 0.795
📊 Specie: Sylvia atricapilla_Eurasian Blackcap, Best Threshold: 1.000, F1-score: 0.792
📊 Specie: Fringilla coelebs_Common Chaffinch, Best Threshold: 0.848, F1-score: 0.670
📊 Specie: Troglodytes troglodytes_Eurasian Wren, Best Threshold: 0.384, F1-score: 0.964
📊 Specie: Muscicapa striata_Spotted Flycatcher, Best Threshold: 0.010, F1-score: 0.965
📊 Specie: Glaucidium passerinum_Eurasian Pygmy-Owl, Best Threshold: 0.010, F1-score: 1.000
📊 Specie: Pyrrhula pyrrhula_Eurasian Bullfinch, Best Threshold: 0.010, F1-score: 0.889
📊 Specie: Periparus ater_Coal Tit, Best Threshold: 0.263, F1-score: 0.977
📊 Specie: Lophophanes cristatus_Crested Tit, Best Threshold: 0.040, F1-score: 0.761
📊 Specie: Regulus regulus_Goldcrest, Best Threshold: 0.051, F1-score: 0.969
📊 Specie: Turdus merula_Eurasian Blackbird, Best Threshold: 0.141, F1-score: 0.960
📊 Specie: Certhia familiaris_Eurasian Treecreeper, Best Threshold: 0.01

# Test Model

In [118]:
from sklearn.metrics import average_precision_score, precision_recall_fscore_support

def compute_samplewise_mAP(y_true, y_probs):
    """
    Calcola la sample-wise mAP (media delle AP per ogni sample).
    """
    ap_per_sample = []
    for i in range(y_true.shape[0]):
        if np.sum(y_true[i]) == 0:
            continue  # Evita sample senza label positive
        ap = average_precision_score(y_true[i], y_probs[i])
        ap_per_sample.append(ap)
    return np.mean(ap_per_sample)

def compute_classwise_mAP(y_true, y_probs):
    """
    Calcola la class-wise mAP (media delle AP per ogni classe).
    """
    ap_per_class = []
    for i in range(y_true.shape[1]):
        if np.sum(y_true[:, i]) == 0:
            continue  # Evita classi mai presenti
        ap = average_precision_score(y_true[:, i], y_probs[:, i])
        ap_per_class.append(ap)
    return np.mean(ap_per_class)

def compute_f05(y_true, y_pred):
    _, _, f05, _ = precision_recall_fscore_support(
        y_true, y_pred, beta=0.5, average='macro', zero_division=0
    )
    return f05

In [119]:
import numpy as np
from sklearn.metrics import classification_report
import csv


def test_model(model, dataset_config, batch_size=100, thresholds=0.5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\n🧬 Advanced testing on: {device}")

    test_loader = utils.get_dataloader(dataset_config, split="test", batch_size=batch_size, shuffle=False)
    model.eval()
    criterion = nn.BCEWithLogitsLoss()
    class_names = list(dataset_config['mappings'].keys())
    total_loss = 0.0
    all_preds = []
    all_probs = []
    all_labels = []

    use_custom_threshold = isinstance(thresholds, dict)

    with torch.no_grad():
        for mel_spec, labels, _ in test_loader:
            mel_spec = mel_spec.to(device)
            labels = labels.to(device)

            outputs = model(mel_spec)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            probs = torch.sigmoid(outputs)

            if use_custom_threshold:
                batch_preds = torch.zeros_like(probs)
                for i, class_name in enumerate(class_names):
                    thresh = thresholds.get(class_name, 0.5)
                    batch_preds[:, i] = (probs[:, i] > thresh).float()
            else:
                batch_preds = (probs > thresholds).float()

            all_probs.append(probs.cpu())
            all_preds.append(batch_preds.cpu())
            all_labels.append(labels.cpu())

    avg_loss = total_loss / len(test_loader)
    all_probs = torch.cat(all_probs).numpy()
    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    samplewise_map = compute_samplewise_mAP(all_labels, all_probs)  # chiamata mAP
    classwise_map = compute_classwise_mAP(all_labels, all_probs)    # chiamata cmAP
    f05_score = compute_f05(all_labels, all_preds)

    with open(f"models/{MODEL_NAME}/metrics_output.csv", mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Metric", "Value"])
        writer.writerow(["mAP (sample-wise)", samplewise_map])
        writer.writerow(["cmAP (class-wise)", classwise_map])
        writer.writerow(["F0.5 Score", f05_score])

    # 👇 Report
    clf_report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True, zero_division=0)
    return avg_loss, clf_report, samplewise_map, classwise_map, f05_score

In [120]:
avg_loss, clf_report, samplewise_map, classwise_map, f05_score = test_model(model, dataset_config, thresholds=best_thresholds)


🧬 Advanced testing on: cuda


In [121]:
print("mAP Score: ", samplewise_map)
print("mcAP Score: ", classwise_map)
print("F0.5 Score: ", f05_score)

mAP Score:  0.46221953535410715
mcAP Score:  0.23051246445948737
F0.5 Score:  0.17216839460921807


In [122]:
from io import StringIO
import pandas as pd

clf_report_df = pd.read_json(StringIO(json.dumps(clf_report)), orient='index')
clf_report_df

,precision,recall,f1-score,support
Regulus ignicapilla_Common Firecrest,0.436885,0.647631,0.521782,823
Sylvia atricapilla_Eurasian Blackcap,0.000000,0.000000,0.000000,1445
Fringilla coelebs_Common Chaffinch,0.628083,0.122502,0.205017,2702
Troglodytes troglodytes_Eurasian Wren,0.600000,0.020134,0.038961,149
Muscicapa striata_Spotted Flycatcher,1.000000,0.094340,0.172414,159
Glaucidium passerinum_Eurasian Pygmy-Owl,0.000000,0.000000,0.000000,27
Pyrrhula pyrrhula_Eurasian Bullfinch,0.400000,0.129032,0.195122,31
Periparus ater_Coal Tit,0.250000,0.013245,0.025157,151
Lophophanes cristatus_Crested Tit,0.000000,0.000000,0.000000,43
Regulus regulus_Goldcrest,0.217822,0.309859,0.255814,142


In [123]:
with open(f'{MODEL_PATH}/classification_report.json', 'w') as f:
    json.dump(clf_report, f)